In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import requests
import pandas as pd
from src.tables import get_machines
from src.utils import df_na_vals

In [31]:
raw = get_machines()

In [32]:
df_na_vals(raw)

bw_nvlink                 ->          Missing values: 0 (0.00%)
compute_cap               ->          Missing values: 0 (0.00%)
cpu_cores                 ->          Missing values: 6 (0.48%)
cpu_name                  ->          Missing values: 11 (0.88%)
cpu_ram                   ->          Missing values: 0 (0.00%)
credit_balance            ->          Missing values: 1246 (100.00%)
credit_discount           ->          Missing values: 1246 (100.00%)
credit_discount_max       ->          Missing values: 213 (17.09%)
cuda_max_good             ->          Missing values: 0 (0.00%)
direct_port_count         ->          Missing values: 0 (0.00%)
discount_rate             ->          Missing values: 213 (17.09%)
discounted_dph_total      ->          Missing values: 0 (0.00%)
discounted_hourly         ->          Missing values: 0 (0.00%)
disk_bw                   ->          Missing values: 1 (0.08%)
disk_name                 ->          Missing values: 0 (0.00%)
disk_space             

['credit_balance', 'credit_discount']

In [33]:
cpu_name = raw.cpu_name.str.replace(r'^\d\dth Gen ','')
cpu_name = cpu_name.str.replace(r'^AMD','')
cpu_name = cpu_name.str.replace(r'Threadripper','TR')
cpu_name = cpu_name.str.replace(r'Processor$','')
cpu_name = cpu_name.str.replace(r'-Core','C')
cpu_name = cpu_name.str.replace(r' with Radeon Graphics','')
# cpu_name = raw.cpu_name.str.replace('Core™', '')
raw.cpu_name = cpu_name
raw['cores'] = (raw.cpu_cores / raw.num_gpus).fillna(0).astype(int)
raw['ram'] = (raw.cpu_ram / raw.num_gpus / 1024).round().astype(int)
raw['dph'] = raw.dph_base / raw.num_gpus
raw['disk'] = (raw.disk_space / raw.num_gpus).astype(int)
raw['rented_fraq'] = '[' + raw.num_gpus_rented.astype(str) + "/" + raw.num_gpus.astype(str) + ']'
raw['country'] = raw.location.dropna().apply(lambda loc: loc['country'])
# raw['rented'] = raw.num_gpus_rented > 0
raw['rented'] = raw.num_gpus_rented == raw.num_gpus
raw['min_fraq'] = raw.min_chunk/raw.num_gpus

raw.dlperf = raw.dlperf.round(1)
raw.dlperf_per_dphtotal = raw.dlperf_per_dphtotal.round(1)
raw.disk_bw = raw.disk_bw.round()
raw.duration = pd.to_timedelta(raw.duration * 1e9).dt.days
# raw.end_date = (pd.to_datetime(raw.end_date * 1e9) - pd.Timestamp.now()).dt.days
raw.end_date = pd.to_datetime(raw.end_date * 1e9)
raw.start_date = pd.to_datetime(raw.start_date * 1e9)
raw.set_index('machine_id', inplace=True)

C:\Users\ergot\AppData\Local\Temp\ipykernel_34192\1583430346.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = raw.cpu_name.str.replace(r'^\d\dth Gen ','')
C:\Users\ergot\AppData\Local\Temp\ipykernel_34192\1583430346.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = cpu_name.str.replace(r'^AMD','')
C:\Users\ergot\AppData\Local\Temp\ipykernel_34192\1583430346.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = cpu_name.str.replace(r'Processor$','')


In [36]:
cols = ['gpu_name',  'rented_fraq', 'dph', 'credit_discount_max', 'dlperf_per_dphtotal', 'cores', 'ram', 'disk', 'disk_bw',
         'min_chunk', 'country', 'storage_cost', 'reliability2', 'duration', 'inet_up', 'inet_down', 'cuda_max_good', 'driver_version',  'pci_gen', 'gpu_lanes', 'hosting_type',  'pcie_bw', 'cpu_name', 'mobo_name',  'location']

raw.query('dph >= 0.5')[cols].sort_values(by='dph', ascending=False)

,gpu_name,rented_fraq,dph,credit_discount_max,dlperf_per_dphtotal,cores,ram,disk,disk_bw,min_chunk,...,inet_down,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,location
machine_id,,,,,,,,,,,,,,,,,,,,,
11305,RTX 3090,[1/1],13.00,0.40,2.5,16,31,827,2742.0,1,...,87.0,12.1,530.41.03,4,16,NaN,23.3,Core™ i5-12600K,B760 Pro RS,"{'country': 'US', 'location': 'Moses Lake, Was..."
6767,RTX A6000,[0/2],5.01,0.40,5.2,10,126,1215,2395.0,1,...,40.9,12.0,525.125.06,3,16,NaN,11.7,Core™ i9-10900X,WS X299 SAGE,"{'country': 'CA', 'location': 'Vancouver, Brit..."
6063,A100 SXM4,[0/8],2.00,0.40,10.4,32,126,188,546.0,4,...,329.4,12.0,525.85.12,4,16,NaN,25.1,EPYC 7763 64C,H12DGO,"{'country': 'US', 'lat': 37.751, 'long': -97.8..."
11877,RTX 4090,[0/1],2.00,0.15,39.5,96,63,6642,2595.0,1,...,180.7,12.0,525.125.06,4,16,NaN,22.0,EPYC 7642 48C,S8030GM2NE,"{'country': 'CZ', 'location': 'Drasov, Okres B..."
11880,A100 SXM4,[1/1],2.00,0.50,19.5,16,63,548,3349.0,1,...,868.3,12.2,535.54.03,4,16,NaN,25.4,Ryzen 7 5800X 8C,B550 Phantom Gaming 4,"{'country': 'CA', 'location': 'Quebec', 'lat':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9077,RTX 4090,[8/8],0.50,0.40,86.2,32,63,175,2189.0,4,...,463.2,12.0,525.105.17,4,16,NaN,22.6,EPYC 7742 64C,ROME2D32GM,"{'country': 'TW', 'location': 'Taipei, Taipei ..."
9101,RTX 4090,[2/2],0.50,0.10,131.0,12,63,1665,1305.0,1,...,759.4,12.0,525.116.04,3,16,NaN,12.5,Ryzen TR 1920X 12C,X399 AORUS PRO,"{'country': 'RO', 'location': 'Bucharest, Bucu..."
10391,RTX 4090,[2/2],0.50,0.40,138.0,12,63,715,3472.0,1,...,305.9,12.0,525.105.17,4,8,NaN,12.7,Ryzen 9 7900X 12C,ProArt B650,"{'country': 'JP', 'location': 'Tokyo', 'lat': ..."


In [35]:
raw[cols].query('country == "RU"')

,gpu_name,rented_fraq,cuda_max_good,driver_version,dph,credit_discount_max,dlperf_per_dphtotal,cores,ram,disk,...,reliability2,duration,inet_up,inet_down,pci_gen,hosting_type,pcie_bw,cpu_name,mobo_name,location
machine_id,,,,,,,,,,,,,,,,,,,,,
11637,RTX 3080 Ti,[3/3],12.0,525.125.06,0.150,0.0,165.0,10,21,521,...,0.941792,29.0,401.0,437.4,4,NaN,6.0,Ryzen 9 5950X 16C,ROG STRIX X570,"{'country': 'RU', 'location': 'Maslyaninskiy, ..."
11613,RTX 3080 Ti,[1/2],12.0,525.125.06,0.150,0.0,166.2,18,16,677,...,0.966970,29.0,406.7,430.4,3,NaN,11.7,Xeon® E5-2696 v3,X99,"{'country': 'RU', 'location': 'Maslyaninskiy, ..."
11536,RTX 3090,[4/4],12.0,525.105.17,0.230,0.1,128.6,14,63,361,...,0.994424,NaN,699.9,753.6,3,NaN,10.2,Xeon® E5-2680 v4,Z10PG,"{'country': 'RU', 'location': 'Moscow Oblast',..."
11480,RTX 3090,[2/2],12.0,525.125.06,0.175,0.0,165.9,16,63,209,...,0.991865,28.0,189.8,205.0,3,NaN,6.4,Ryzen 9 5950X 16C,ROG STRIX X570,"{'country': 'RU', 'location': 'Maslyaninskiy, ..."
11395,RTX 3090,[4/4],12.0,525.125.06,0.230,0.1,129.5,14,63,243,...,0.994547,24.0,674.0,709.9,3,NaN,10.6,Xeon® E5-2680 v4,Z10PG,"{'country': 'RU', 'location': 'Moscow Oblast',..."
11385,RTX 4090,[6/6],12.0,525.105.17,0.395,0.4,150.2,10,42,191,...,0.966162,3.0,419.7,535.5,4,NaN,23.3,EPYC 7532 32C,ROMED8,"{'country': 'RU', 'location': 'Moscow', 'lat':..."
11230,RTX 4090,[2/2],12.0,525.105.17,0.400,0.2,201.9,64,63,818,...,0.993515,1.0,539.7,480.3,4,NaN,23.1,EPYC 7B13 64C,ROMED8,"{'country': 'RU', 'location': 'Fryazino, Mosco..."
10891,A100 PCIE,[0/8],12.0,525.105.17,0.800,0.4,14.5,1,118,13,...,0.971270,NaN,469.6,671.6,4,NaN,15.3,EPYC 7H12 64C,440BX Desktop Reference Platform,"{'country': 'RU', 'lat': 55.7386, 'long': 37.6..."
10794,RTX 3090,[8/8],12.0,525.116.04,0.400,0.4,73.0,12,63,440,...,0.997459,660.0,529.4,652.3,4,NaN,24.2,EPYC 7402 24C,H12DSG,"{'country': 'RU', 'lat': 55.7386, 'long': 37.6..."
